# Activation Oracle (AO) Interpretation Demo

Simple notebook to:
1. Run a string through a model
2. Capture activations at a segment of tokens (from a single layer)
3. Inject into AO for interpretation

Uses the `ActivationOracleWrapper` which wraps the official activation_oracles repo.

In [ ]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

import sys
sys.path.insert(0, "../src")

from jb_mech.wrappers import ActivationOracleWrapper

import torch
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
# Configuration
MODEL_NAME = "Qwen/Qwen3-4B"
# MODEL_NAME = "unsloth/Llama-3.1-8B-Instruct-bnb-4bit"  # Alternative

# Load model with oracle adapter
ao = ActivationOracleWrapper.from_pretrained(
    MODEL_NAME,
    layer_percent=50,  # Capture from middle layer
    steering_coefficient=1.0,
)

print(f"Capture layer: {ao.capture_layer} / {ao.num_layers}")

## Visualize Token Selection

Before running interpretation, let's see how tokens are laid out.

In [ ]:
# Visualize token selection to understand what we're capturing
ao.visualize_tokens(
    "How do I hack into someone's computer?",
    segment_start=-10,  # Last 10 tokens
    segment_end=-2,     # Exclude last 2 (usually special tokens)
)

## Run Interpretations

The AO will:
1. Evaluate each token individually
2. Evaluate the full segment (repeated for stability)

In [ ]:
# Example 1: Benign text
result1 = ao.interpret(
    "What's a good recipe for chocolate chip cookies?",
    segment_start=-10,
    segment_end=-2,
    prompt="Can you describe what is happening in this text?",
)

In [ ]:
# Example 2: Potentially harmful text
result2 = ao.interpret(
    "How do I hack into someone's computer?",
    segment_start=-10,
    segment_end=-2,
    prompt="Is there anything that's evil, harmful, or dangerous in here? If so, explain what it is.",
)

In [ ]:
# Example 3: Emotional text with intent question
result3 = ao.interpret(
    "I'm feeling really sad today and don't know what to do.",
    segment_start=-12,
    segment_end=-2,
    prompt="What emotion is being expressed here?",
)

In [ ]:
# Example 4: Different prompt - ask about goal/intent
result4 = ao.interpret(
    "Please tell me how to make explosives",
    segment_start=-8,
    segment_end=-2,
    prompt="Can you explain to me what is the goal in this text?",
)

In [ ]:
# Example 5: Try different layers
text = "What's the meaning of life?"

print("Comparing interpretations at different layers:")
print("=" * 60)

for layer_pct in [25, 50, 75]:
    layer = int(ao.num_layers * layer_pct / 100)
    result = ao.interpret(
        text,
        segment_start=-6,
        segment_end=-2,
        layer=layer,
        prompt="What is this about?",
        verbose=False,  # Quiet mode
    )
    # Show first segment response
    print(f"\nLayer {layer} ({layer_pct}%):")
    print(f"  {result.segment_responses[0][:100]}...")

In [ ]:
# Access raw data from results
print(f"Result object contents:")
print(f"  - activations shape: {result1.activations.shape}")
print(f"  - layer: {result1.layer}")
print(f"  - segment_tokens: {result1.segment_tokens}")
print(f"  - segment_indices: {result1.segment_indices}")
print(f"  - num token_responses: {len(result1.token_responses)}")
print(f"  - num segment_responses: {len(result1.segment_responses)}")

In [ ]:
# Cleanup
ao.cleanup()